In [3]:
# 04_indicator_thresholds.ipynb (or a .py you run once)

import pandas as pd, numpy as np, json

df = pd.read_csv("../data/ai4i2020.csv")
df["Temp diff [K]"] = df["Process temperature [K]"] - df["Air temperature [K]"]
df["power_index"]   = df["Torque [Nm]"] * df["Rotational speed [rpm]"]
df["wear_torque"]   = df["Tool wear [min]"] * df["Torque [Nm]"]

def best_thresh_for_greater(score_series, y, candidates):
    # choose threshold t for (score > t) that maximizes F1
    from sklearn.metrics import f1_score
    best_t, best_f1 = None, -1
    for t in candidates:
        pred = (score_series > t).astype(int)
        f1 = f1_score(y, pred, zero_division=0)
        if f1 > best_f1:
            best_f1, best_t = f1, t
    return best_t, best_f1

def best_band(power, y, lows, highs):
    # choose (low, high) for (power < low) | (power > high)
    from sklearn.metrics import f1_score
    best = (None, None, -1)
    for lo in lows:
        for hi in highs:
            if lo >= hi: 
                continue
            pred = ((power < lo) | (power > hi)).astype(int)
            f1 = f1_score(y, pred, zero_division=0)
            if f1 > best[2]:
                best = (lo, hi, f1)
    return best

# candidate grids (feel free to make denser)
q = np.linspace(0.05, 0.95, 19)

hdf_t, _ = best_thresh_for_greater(df["Temp diff [K]"], df["HDF"], df["Temp diff [K]"].quantile(q))
osf_t, _ = best_thresh_for_greater(df["wear_torque"], df["OSF"], df["wear_torque"].quantile(q))
twf_t, _ = best_thresh_for_greater(df["Tool wear [min]"], df["TWF"], df["Tool wear [min]"].quantile(q))

lo_cands = df["power_index"].quantile(np.linspace(0.01, 0.40, 15))
hi_cands = df["power_index"].quantile(np.linspace(0.60, 0.99, 15))
pwf_lo, pwf_hi, _ = best_band(df["power_index"], df["PWF"], lo_cands, hi_cands)

cfg = {
    "HDF_dt_k": float(hdf_t),
    "OSF_wear_torque": float(osf_t),
    "TWF_wear": float(twf_t),
    "PWF_low": float(pwf_lo),
    "PWF_high": float(pwf_hi)
}
with open("indicator_thresholds.json", "w") as f:
    json.dump(cfg, f, indent=2)
print("Saved notebooks/indicator_thresholds.json:", cfg)


Saved notebooks/indicator_thresholds.json: {'HDF_dt_k': 8.399999999999977, 'OSF_wear_torque': 9372.904999999999, 'TWF_wear': 206.04999999999927, 'PWF_low': 36641.248999999996, 'PWF_high': 84237.13}
